# Python勉強会 （第1回）

今回の内容は研究室の新人研修（Python）くらいは理解できている前提です．  
分からなくなったら，新人研修の資料とかを読み返してみてください．  
今回の内容がみんなの研究のあまり本質では無いところとかで密かに役立ってくれたりすると幸いです．

## 目次
- pythonを推す理由
- シーケンス型の扱い
  - 内包表現
  - Lambda式
  - 変数の保存
- Pythonでのデバッグ
  - printデバッグ
  - 例外処理
  - pdb
- 研究室でよく使用するライブラリたち
  - numpy
  - scipy
  - cv2(opencv)
  - matplotlib
  - pandas (おまけ)

## pythonを推す理由

### なぜpythonか

> Python（パイソン）は、汎用のプログラミング言語である。コードがシンプルで扱いやすく設計されており、C言語などに比べて、さまざまなプログラムを分かりやすく、少ないコード行数で書けるといった特徴がある．  
核となる本体部分は必要最小限に抑えられている。一方で標準ライブラリやサードパーティ製のライブラリ、関数など、さまざまな領域に特化した豊富で大規模なツール群が用意され、インターネット上から無料で入手でき、自らの使用目的に応じて機能を拡張してゆくことができる。  
Pythonはオブジェクト指向、命令型、手続き型、関数型などの形式でプログラムを書くことができる。動的型付け言語であり、参照カウントベースの自動メモリ管理（ガベージコレクタ）を持つ。
[Wikipedia](https://ja.wikipedia.org/wiki/Python "Python wiki")  

Pythonでは処理を直感的に記述出来て，たくさんのライブラリをただで使える！

### 純粋なpython環境でもたくさんのことができる

pythonは，動的型付け言語なので変数の型を気にすることがない．  
それに加えてlist, dictなどシーケンス型の変数がすごく便利である．
しかも，これらの変数を簡単に書き出し，読み込みをすることができる．
以下に書き出しの例を示す．

In [98]:
import json
data = {
    "研究室メンバー": ["Yamazaki", "Nagahama", "Arnold",
                "Matsumoto",
                "Kamal", "Koishihara", "Shimakawa",
                "Baku", "Sano", "Tanaka", "Tsuda", "Matsuda",
                "Demura", "Higashide", "Kawagoshi", "Nakajima", "Nogami"
               ],
    "ロボット": ["HIRO016", "HIRO023", "Baxter", "HSR", "icart-mini", "Roomba","NAOs"],
    "Room": [410, 406, 405, 103, 108],
    }
print(data)

# 書き出し
with open("sample.json", "w") as f:
    json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))
    
# 読み込み
with open("sample.json", "r") as f:
    read_data = json.load(f)
print(read_data)

{'研究室メンバー': ['Yamazaki', 'Nagahama', 'Arnold', 'Matsumoto', 'Kamal', 'Koishihara', 'Shimakawa', 'Baku', 'Sano', 'Tanaka', 'Tsuda', 'Matsuda', 'Demura', 'Higashide', 'Kawagoshi', 'Nakajima', 'Nogami'], 'ロボット': ['HIRO016', 'HIRO023', 'Baxter', 'HSR', 'icart-mini', 'Roomba', 'NAOs'], 'Room': [410, 406, 405, 103, 108]}
{'Room': [410, 406, 405, 103, 108], 'ロボット': ['HIRO016', 'HIRO023', 'Baxter', 'HSR', 'icart-mini', 'Roomba', 'NAOs'], '研究室メンバー': ['Yamazaki', 'Nagahama', 'Arnold', 'Matsumoto', 'Kamal', 'Koishihara', 'Shimakawa', 'Baku', 'Sano', 'Tanaka', 'Tsuda', 'Matsuda', 'Demura', 'Higashide', 'Kawagoshi', 'Nakajima', 'Nogami']}


In [104]:
hoge = {0: "fuga", 2: "piyo"}
hoge[2]

'piyo'

### ライブラリが豊富

最近流行りのディープラーニング関係のライブラリはほとんどがpythonのAPIが存在する．
- Tensorflow
- Chainer
- Caffe
など

ほかにも
数値計算系ではnumpyやscipy, matplotlib,
画像処理ではopencvやPILやPillowなど，
便利なライブラリがたくさんある．  
細かい実装はこれらのライブラリに任せて我々はやりたいことのコードを書くだけでよい．  
Pythonではこれらのライブラリの扱いに慣れることで面倒な部分のコーディングに時間を書けなくて済む．

---

## シーケンス型の扱い

先程の話でもあったが，pythonではlistとdictに代表されるシーケンス型の変数が非常に便利で強力である．

シーケンス型（list & dict）の特徴
- 特に何も考えずに変数を蓄えられる.
    - 長さが可変
    - 要素に型の制限は無い
        - listのlistなどの入れ子にすることもできる
    
- 保存と読込みが簡単に行える．
    - 学習データの収集に使える．
    - 要素に種類がある場合はlistではなく,dictを使うことを薦める．
        - 他人とデータを共有するときに，いろいろとやりやすい．
        
- 高階関数により，一括して処理ができる．
    - map, filter, reduceによる処理
    - sortedによる要素のソート


以下ではこれらの変数の扱いについて紹介する．

### 内包表現

内包表現とは，リストや辞書，ジェネレータなどのiterableオブジェクトのループ処理を単純に書く記法のことである．それに対応して，リスト内包，辞書内包，などと呼ばれる．
今回は多分よく使うであろう，リスト内包と辞書内包について紹介していきます．

リストの処理はPythonの基本処理の中でも結構重いものに当たる．

1. for文＋list.append()
    - 読みやすい？
    - 要素数がある程度大きなリストを作る場合かなり時間がかかってしまう．  
1. 内包表現を使用する
    - 方法１よりも高速
    - 慣れないと若干読みづらい    
1. C++で書いて，Pythonで呼び出す．
    - やったことないのでわかりません．
    - 一番高速化できるらしいです．

内包表現では条件をつけたりして，ある程度複雑なリストの作成も可能だが，あんまり複雑な場合ただただ読みづらいだけになる可能性もある．  
リストがある程度複雑な場合は，for分とかif分とか使って書いた方がいいかもしれないです．  
そこら辺は可読性と実効速度のどちらを優先するのかによって変わってくるので，どちらが正しいとも言えない．


とりあえず，例題を書いてみる.  
以下は1から1,000,000までのリストをfor文を使用して作成する方法と内包表現を用いて作成するコードである．

#### for文 + list.append()

In [105]:
%%timeit
a = []
for i in range(1000000): a.append(i)

10 loops, best of 3: 109 ms per loop


#### 内包表現を使用

In [106]:
%%timeit
a = [i for i in range(1000000)]

10 loops, best of 3: 61.4 ms per loop


今回だと，40ms程差が出ているが，この差が意外と効いてくる．  
この処理を100回繰り返すと4秒，10000回繰り返すと，400秒=約7分も差が出る．  
このくらい単純なリストの場合は内包表現を使用することをおすすめする．

##### 初期値が3，公差が2，長さが30の等差数列

In [107]:
a = [3 + i*2 for i in range(30)]
a[:10], len(a)

([3, 5, 7, 9, 11, 13, 15, 17, 19, 21], 30)

内包表現内でif文も取り扱うことができる．  
以下に例題を示していくが，if分くらいならまだ大丈夫だが，
elseまで入れると内包表現は非常に読みづらいものになる．

以下は0から100まででの3の倍数のリストを作成するコード

#### for文 + if文 + list.append()

In [4]:
a = []
for i in range(100):
    if i % 3 == 0:
        a.append(i) 
a[:10]

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

#### 内包表現を使用

In [63]:
a = [i for i in range(100) if i % 3 == 0]
a[:10]

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

else文まで入るとかなり読みづらい...

以下は0から100までの3の倍数はそのまま，それ以外は0としたリストを作成するコード

#### for文 + if-else文 + list.append()

In [60]:
a = []
for i in range(100):
    if i % 3 == 0:
        a.append(i) 
    else:
        a.append(0)
a[:10]

[0, 0, 0, 3, 0, 0, 6, 0, 0, 9]

#### 内包表現を使用

In [111]:
a = [i if i % 3 == 0 else 0  for i in range(100)]
a[:10]

[0, 0, 0, 3, 0, 0, 6, 0, 0, 9]

In [109]:
i = True if 10 > 1 else False
i

True

これくらいになると読めなくは無いが，読むのにとても時間がかかるのでおすすめしない

In [74]:
a = [[j**i if j % 3 == 0 else j for j in range(10)] for i in range(10) if i % 2 == 0]
a

[[1, 1, 2, 1, 4, 5, 1, 7, 8, 1],
 [0, 1, 2, 9, 4, 5, 36, 7, 8, 81],
 [0, 1, 2, 81, 4, 5, 1296, 7, 8, 6561],
 [0, 1, 2, 729, 4, 5, 46656, 7, 8, 531441],
 [0, 1, 2, 6561, 4, 5, 1679616, 7, 8, 43046721]]

リスト内包についてはここまでです．  
あとは辞書内包表現について少しだけ説明して行きます．
辞書オブジェクトは他の人とデータを共有するときにおすすめのデータ型なので，辞書内包についても覚えておいて損はないと思います．
内容としてはリスト内包が分かれば問題なくわかると思います．

### 例題

３ｘ３の行列  
[1 2 3]  
[4 5 6]  
[7 8 9]  

In [124]:
hoge = [[1, 2, 3],
[4, 5, 6],
[7, 8, 9]]
print(hoge)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [129]:
set([c for c in "hogehgoeohgeoijfeoawjef;oiajwe;of19201923iohgeo"])

{'0', '1', '2', '3', '9', ';', 'a', 'e', 'f', 'g', 'h', 'i', 'j', 'o', 'w'}

#### 適当な辞書内包の例

In [137]:
a = {k: v for k,v in zip("hoge", range(4))}

b = {kv[1]: kv[0] for kv in zip("hoge", range(4))}
a, b

({'e': 3, 'g': 2, 'h': 0, 'o': 1}, {0: 'h', 1: 'o', 2: 'g', 3: 'e'})

In [138]:
list(zip("hoge", range(5)))

[('h', 0), ('o', 1), ('g', 2), ('e', 3)]

In [140]:
a = {k: v for k,v in zip("hogehoge", range(8))}
a

{'e': 7, 'g': 6, 'h': 4, 'o': 5}

#### if文, if-else文を使用する例

In [106]:
a = {k: v for k, v in zip("hogefuga", range(8)) if k != 'g'}
a

{'a': 7, 'e': 3, 'f': 4, 'h': 0, 'o': 1, 'u': 5}

In [142]:
a = {k: v if v % 2 == 0 else 0 for k, v in zip("hogebar", range(7))}
a

{'a': 0, 'b': 4, 'e': 0, 'g': 2, 'h': 0, 'o': 0, 'r': 6}

In [111]:
a = {k: v if v % 2 == 0 else 0 for k, v in zip("hogefuga", range(8)) if k != "g"}
a

{'a': 0, 'e': 0, 'f': 4, 'h': 0, 'o': 0, 'u': 0}

In [148]:
a = {i*2+1: v**2 if v % 2 == 0 else v*2 for i, v in enumerate(range(10))}
a

{1: 0, 3: 2, 5: 4, 7: 6, 9: 16, 11: 10, 13: 36, 15: 14, 17: 64, 19: 18}

In [147]:
list(enumerate("hogehoge"))

[(0, 'h'),
 (1, 'o'),
 (2, 'g'),
 (3, 'e'),
 (4, 'h'),
 (5, 'o'),
 (6, 'g'),
 (7, 'e')]

あとはジェネレータ内包とか，セット内包とかがあるけど，基本的にはリスト内包が理解できれば他のものも理解できると思われる．  
途中出てくるzipとかenumerateが分からない場合はググってください！

### Lambda式

無名関数のことである．関数を変数として扱える．map, filter, reduceなどが代表に挙げられる高階関数を使用するときに役に立つ．  
ファイル名を扱うような単純な関数としても意外と使える．  
あとはsortedがかなり役に立つ．

Lambda式とは，無名関数のことである．名前の無い関数のことである．  
使いみちの一つの例として，関数を引数とした関数の作成が可能になる．  
言葉ではあまりよく分からないと思われるので，とりあえず例題を幾つか上げる．

mapを使用した例．  
mapの解説を簡単にしておくと，第一引数として入力する関数を第二引数全てに対して適応するような関数である．  
python3の場合，その結果を出力するジェネレータが作成される．この辺の細かい話は今回は割愛する．

In [155]:
f = lambda x: x+3
f(3)

6

In [151]:
a = [i for i in range(100)]
b = map(lambda x:x**2, a)
list(b)[:10]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [153]:
list(map(lambda x:x**2, [i for i in range(100)]))[:10]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [150]:
a = [i for i in range(100)]
def hoge(x): return x**2

b = map(hoge, a)
list(b)[:10]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

上の例の意味としては，まず0から99までのリストを作成する．（内包表現）
次にそのリストの各要素に対して，引数1の関数を適応し，その関数の出力を新しくリストとしてまとめる（本当はジェネレータ）．
これを引数の関数を変更すると，当然であるが出力するリストの中身も変化する．

In [76]:
a = [i for i in range(100)]
b = map(lambda x:x**3, a)
list(b)[:10]

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]

In [158]:
f = (lambda x:x**i for i in range(10))
x = [func(4) for func in f]
x

[1, 4, 16, 64, 256, 1024, 4096, 16384, 65536, 262144]

上の例はLambda式のリスト（ほんとはジェネレータ）を作成し，それぞれの関数を適応した結果のリストを作成している．  
あんまりいい例では無いがこんな繰り返す処理を行うのに何度も関数を作る必要がなくなる．

sortedを使用した例．  
sortedは第一引数のシーケンスをkeyに従って並び替え，その結果を出力する高階関数である．  
listのlistや，tuple，classのソートに使用する．  
keyにはlambda式で比べる要素を記述するとよい

#### listのlistをソート

In [164]:
import random, pprint
a = [[random.randint(1, 100) for j in range(3)] for i in range(10)]
print("a:"); pprint.pprint(a); print()
pprint.pprint([sorted(a, key=lambda x:x[i]) for i in range(3)])

a:
[[4, 30, 84],
 [43, 47, 21],
 [69, 79, 37],
 [51, 93, 36],
 [3, 8, 20],
 [98, 34, 83],
 [58, 14, 9],
 [52, 20, 95],
 [38, 35, 33],
 [58, 29, 49]]

[[[3, 8, 20],
  [4, 30, 84],
  [38, 35, 33],
  [43, 47, 21],
  [51, 93, 36],
  [52, 20, 95],
  [58, 14, 9],
  [58, 29, 49],
  [69, 79, 37],
  [98, 34, 83]],
 [[3, 8, 20],
  [58, 14, 9],
  [52, 20, 95],
  [58, 29, 49],
  [4, 30, 84],
  [98, 34, 83],
  [38, 35, 33],
  [43, 47, 21],
  [69, 79, 37],
  [51, 93, 36]],
 [[58, 14, 9],
  [3, 8, 20],
  [43, 47, 21],
  [38, 35, 33],
  [51, 93, 36],
  [69, 79, 37],
  [58, 29, 49],
  [98, 34, 83],
  [4, 30, 84],
  [52, 20, 95]]]


In [167]:
b = sorted(a, key=lambda x:x[0]+x[1]+x[2])
b

[[3, 8, 20],
 [58, 14, 9],
 [38, 35, 33],
 [43, 47, 21],
 [4, 30, 84],
 [58, 29, 49],
 [52, 20, 95],
 [51, 93, 36],
 [69, 79, 37],
 [98, 34, 83]]

#### classのソート

In [171]:
class hoge(object):
    def __init__(self, n):
        self.num = n
    def fuga(self, m=10):
        print(self.num % m)
        return self.num % m
    
a = [hoge(random.randint(1, 100)) for i in range(10)]
print([h.num for h in a])

b = sorted(a, key=lambda x:x.num)
c = sorted(a, key=lambda x:x.fuga())
print([h.num for h in b])
print([h.num for h in c])

[83, 89, 12, 65, 96, 9, 82, 93, 53, 72]
3
9
2
5
6
9
2
3
3
2
[9, 12, 53, 65, 72, 82, 83, 89, 93, 96]
[12, 82, 72, 83, 93, 53, 65, 96, 89, 9]


In [176]:
fname = lambda n1, n2: "hoge_fuga_%02d_%02d.py"%(n1, n2)
["pathpath" + fname(i, j) for i, j in zip(range(1, 11), range(10))]

['pathpathhoge_fuga_01_00.py',
 'pathpathhoge_fuga_02_01.py',
 'pathpathhoge_fuga_03_02.py',
 'pathpathhoge_fuga_04_03.py',
 'pathpathhoge_fuga_05_04.py',
 'pathpathhoge_fuga_06_05.py',
 'pathpathhoge_fuga_07_06.py',
 'pathpathhoge_fuga_08_07.py',
 'pathpathhoge_fuga_09_08.py',
 'pathpathhoge_fuga_10_09.py']

### 変数の保存

pythonではlistやdictなどの変数を保存するための機構が存在する．

保存するデータ

In [91]:
data = {
    "Member": ["Yamazaki", "Nagahama", "Arnold",
                "Matsumoto",
                "Kamal", "Koishihara", "Shimakawa",
                "Baku", "Sano", "Tanaka", "Tsuda", "Matsuda",
                "Demura", "Higashide", "Kawagoshi", "Nakajima", "Nogami"
               ],
    "Robot": ["HIRO016", "HIRO023", "Baxter", "HSR", "icart-mini", "Roomba","NAOs"],
    "Room": [410, 406, 405, 103, 108],
    }
data

{'Member': ['Yamazaki',
  'Nagahama',
  'Arnold',
  'Matsumoto',
  'Kamal',
  'Koishihara',
  'Shimakawa',
  'Baku',
  'Sano',
  'Tanaka',
  'Tsuda',
  'Matsuda',
  'Demura',
  'Higashide',
  'Kawagoshi',
  'Nakajima',
  'Nogami'],
 'Robot': ['HIRO016',
  'HIRO023',
  'Baxter',
  'HSR',
  'icart-mini',
  'Roomba',
  'NAOs'],
 'Room': [410, 406, 405, 103, 108]}

#### jsonによる書出しと読込み

In [180]:
import json

# 書出し
#with open("json_sample.json", "w") as f:
#    json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))
    
# 読込み
with open("json_sample.json", "r") as f:
    json_data = json.load(f)
print(json_data)

{'Member': ['Yamazaki', 'Nagahama', 'Arnold', 'Matsumoto', 'Kamal', 'Koishihara', 'Shimakawa', 'Baku', 'Sano', 'Tanaka', 'Tsuda', 'Matsuda', 'Demura', 'Higashide', 'Kawagoshi', 'Nakajima', 'Nogami'], 'Robot': ['HIRO016', 'HIRO023', 'Baxter', 'HSR', 'icart-mini', 'Roomba', 'NAOs', 'nekonote'], 'Room': [410, 406, 405, 103, 108]}


#### pickleによる書出しと読込み

In [177]:
import pickle
print(data)

# 書出し
with open("pickle_sample.pickle", "wb") as f:
    pickle.dump(data, f)
    
# 読込み
with open("pickle_sample.pickle", "rb") as f:
    pickle_data = pickle.load(f)
print(pickle_data)

{'研究室メンバー': ['Yamazaki', 'Nagahama', 'Arnold', 'Matsumoto', 'Kamal', 'Koishihara', 'Shimakawa', 'Baku', 'Sano', 'Tanaka', 'Tsuda', 'Matsuda', 'Demura', 'Higashide', 'Kawagoshi', 'Nakajima', 'Nogami'], 'ロボット': ['HIRO016', 'HIRO023', 'Baxter', 'HSR', 'icart-mini', 'Roomba', 'NAOs'], 'Room': [410, 406, 405, 103, 108]}
{'研究室メンバー': ['Yamazaki', 'Nagahama', 'Arnold', 'Matsumoto', 'Kamal', 'Koishihara', 'Shimakawa', 'Baku', 'Sano', 'Tanaka', 'Tsuda', 'Matsuda', 'Demura', 'Higashide', 'Kawagoshi', 'Nakajima', 'Nogami'], 'ロボット': ['HIRO016', 'HIRO023', 'Baxter', 'HSR', 'icart-mini', 'Roomba', 'NAOs'], 'Room': [410, 406, 405, 103, 108]}


変数を保存する方法
- pickle
    - メリット：  
        基本的にどんな型の変数でも保存可能  
        バイナリ化するので軽い？
    - デメリット：
        遅い  
        バイナリ化されるので，保存ファイルを見ても何もわからない
- json
    - メリット：  
        保存ファイルがとても見やすい  
    - デメリット：  
        標準では保存できる型が限られる．（辞書、リスト、文字列、整数、浮動小数点数、True、False、None）  

### まとめ

- pythonではlistやdictが非常に便利である．
    - 型を気にしなくていい
    - 長さは可変
- 内包表現によりlistやdictが簡単に記述でき，高速に処理ができる
    - 可読性の低下に注意
- Lambda式+高階関数により，listやdictの全体的な処理を簡潔に記述できる
    - map, reduce, filter
    - sorted
- pythonでは変数の保存＆読込みが簡単にできる
    - json
        - 保存ファイルが読みやすい
    - pickle
        - 基本的にどんな変数でも保存可能

---

## デバッグ

### printデバッグ

多くの人がやっているであろうデバッグ方法  
とりあえずデバッグといえばこの方法が使用されている気がする  
pythonでprintデバッグをする場合は，reprを使用した方がよりわかりやすくなる

In [92]:
a = ["1", "2", "3"]
b = [1, 2, 3]
print(a, b)
print(repr(a), repr(b))

['1', '2', '3'] [1, 2, 3]
['1', '2', '3'] [1, 2, 3]


### 例外処理

例外処理を覚えておくと，バグフィックスに非常に役に立つ．  
あと，予め例外処理がしっかりとなされているライブラリなんかを使うときはこの辺を知っておかないと，エラーで停止が頻発することが予想される．  
本気で入力に対して正しい出力を計算するプログラムを書こうとすると大半の部分が例外処理になる． 

以下でも書いてあるが，この例外処理がしっかりと行われているので，ライブラリでできることはライブラリを使ってやったがいい．

#### エラーを検出する

In [65]:
def exception_sample():
    a = 1
    b = 2
    c = "3"
    try:
        print(a + b)
        print(b * c)
        print(c + a)
    except:
        print("計算できない")
        
exception_sample()

3
33
計算できない


ライブラリを使用しているときにエラーでプログラムが終了するのは大体この例外処理によるものである．  
opencvのmat処理やrosのtf関連の処理で頻発してると思われる．
これらは適切に処理してやることで，不本意なプログラム停止は防ぐことができる．

例外処理は不本意なプログラム停止を発生させることがあるが，これのおかげでデバッグがやりやすくなる．
例外処理はプログラムがのどこで例外が発生は発生し，その内容がどのようなものだったのかという情報（スタックトレース）を含んでいる．  
これを使うことで，自分のプログラムのどの部分で良くない処理をしていたのかが理解できる．
これらの細かい内容はpdbのときに説明する．

例外処理のキーワードとしてpythonでは，  
try, except, raise, finally  
というキーワードが挙げられる．
細かい話は今回は省略するが，これらの処理をうまく使うことで例外処理を適切に行うことができる．  
実際では我々のプログラムでは，使用するライブラリが吐き出してくる例外をうまく処理すればよいので，tryとexceptを覚えておけばok.
c++でも例外処理があるので，pythonで例外処理についてある程度理解しているとC++にも応用が効くかもしれない．

### pdb

pythonは標準でpdbというデバッガを用意している．  
デバッガでできることはブレークポイント，ステップ実行，ステップアウト，変数確認が挙げられる．  
pdbのチートシートはいろいろあるので（[例](https://raw.githubusercontent.com/nblock/pdb-cheatsheet/master/pdb-cheatsheet.png "cheatsheet")），
慣れるまではそれを参照しながらデバッグをしていくと良い．

In [62]:
import pdb
def debug_sample(a):
    pdb.set_trace()
    print(1 + a)
    print(1 - a)
    print(1 * a)
    print(1 / a)

debug_sample(1)
debug_sample(0)

> <ipython-input-62-c3244086c071>(4)debug_sample()
-> print(1 + a)
(Pdb) c
2
0
1
1.0
> <ipython-input-62-c3244086c071>(4)debug_sample()
-> print(1 + a)
(Pdb) c
1
1
0


ZeroDivisionError: division by zero

### テスト

pythonではコードのテストツールが用意されており，書いたコードのテストを行うことができる．[Unittest](http://docs.python.jp/2/library/unittest.html "#unittest.TestCase.assertRaises")

### まとめ

- printデバッグ
    - 一番簡単なデバッグ方法
    - 変数を表示するときには，reprを使用するとよい（文字列と数値の区別）
- 例外処理
    - 例外処理によりプログラムで異常な処理を行った時の対処をすることができる．
    - 例外をうまく処理しないとプログラムが終了するのが，きちんと対応すると役に立つ．
    - 例外処理によってデバッグがしやすくなる．（スタックトレースとか）
- pdb
    - python標準装備のデバッガ
    - ブレークポイント，ステップ実行，ステップアウト，変数確認
    - 基本的にはgdbと同じ
- テスト
    - [Unittest](http://docs.python.jp/2/library/unittest.html "#unittest.TestCase.assertRaises")

---

## 研究室でよく使用するライブラリたち

我々の研究室では，様々な計算を行う必要があるが，それらの実装を助けてくれるライブラリがpythonにはたくさんある．
科学計算を行う人達には必須のライブラリであるscipyファミリーが中心になっている．
あとはビジョンを扱うなら，cv2だし，ROSを使うならrospyとか，tfとかがある．

matplotlibは非常に強力なツールなので覚えておくと，色々と役に立つ．
ただ少しお作法が多い気がするので，ラップしてくれてるライブラリ(seabornなど)を使用するのも有り．

pandas+matplotlibをマスターすることで，excelとは完全におさらばできる！

自分でいろんな計算を実装しようと思うのもいいが，基本的にはこういったライブラリを使用する方がメリットが多い（私は趣味でやってます）．  
慣れないうちはこういったライブラリを使うのに抵抗があるかもしれないが，これらのライブラリは多くの人が使用し，デバッグ，最適化されている．  
それらのライブラリを使って計算して，結果がうまく出ない場合は，基本的には自分が間違っていることになるので全て自実装でやるよりも問題発見も早くなると思われる．
自分で実装していた場合，計算がミスっているのか，データが間違っているのか，
そもそもこの計算ではうまく結果が出ないのかがはっきりしなく，
うまくいかないほんとの原因がどこにあるのか特定が大変になるので，pythonではできるだけライブラリに頼る姿勢がいいと思う．

### numpy

pythonを使って数値計算をする時の基本ライブラリである．
numpy.arrayというオブジェクトを提供してくれ，基本的にはこのオブジェクトを使って何かの計算をするというようなライブラリである．  
numpy.arrayはリストに似たiterableなオブジェクトであるが，メモリの配置がリストとは違い，順番に配置されるので，
イテレーションがリストのそれよりも格段に早くなる．  
しかし，繰り返し計算を行う場合，for文を使用して各要素にアクセスしていると非常に時間がかかる．
そのような場合，うまく同様な行列計算として表現してやると，計算が非常に高速に実行される．（恐らく行列計算についてより最適化が行われている）

### scipy

scipyはnumpy.arrayを使った科学計算のライブラリである．
統計，信号処理，画像処理，...など，研究室で必要そうな計算はほとんど実装されていると思われる．
scikit-learnもここに含めるならば，パターン認識もこれに含まれるので研究室で必要な計算はほとんどライブラリが用意されていることになる．
それらをうまく使いこなせれば，ほんとに必要な部分のコーディングに時間がさけるのでおすすめ．
scipyはほんとにいろんな計算が実装されているので，漁って見るのも面白いと思うよ．

### cv2

opencvのpythonラッパー．意外と使える．
基本的なAPIについては実装されているみたい．(確か松田情報だが，一部未実装の関数もいるらしい)  
cv2では画像はnumpy.arrayで表現される．なので，numpyがある程度使えると非常にとっつきやすいものになる．  
画像処理をオンラインで行いたい場合は，C++でコーディングすることをおすすめする．
cv2はあくまで実験結果をオフラインで何かするというときに使うのがよい．  
あとcv2のimshowを使用するよりも，matplotlibのimshowを使うことをおすすめする．（カラーチャンネルの順番が違うので注意）

### matplotlib

グラフ作成ライブラリ  
主にnumpy.arrayを入力して，様々なグラフを出力してくれる．  
色々と癖があるので，慣れるまでに結構時間がかかるかも．
ただ非常に高機能ではあるので，使いこなせればかなりな武器になるのは間違いない．  
gnuplotというツールもあるらしいが，pythonをつかうならmatplotlibでしょ．

### pandas

データベースを取り合うライブラリ  
excelの行列みたいなデータを作成できるし，csvとして出力したりもできる．  
データに様々なクエリがあったり，次元が多い場合に整理するのに便利だし，わかりやすくなるのではないか．  
pandas.DataFrameはnumpy.arrayと仲がいいらしい．  
上でも書いたが，pandas+matplotlibを使いこなせれば，excelの表計算には完全に勝利できる！．

### 画像に対して，RGBについてのK-meansを行った例（cv2, sklearn, numpy, matplotlib）  
Linux環境で試してください．

In [181]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import cv2
%matplotlib inline

img = cv2.imread("画像データ")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

k = 5
km = KMeans(n_clusters=k, random_state=0).fit(img.reshape((640*480, 3)))
km_img = np.copy(img.reshape((640*480, 3)))

for i in range(k): km_img[km.labels_ == i] = km.cluster_centers_[i]
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(km_img.reshape(img.shape))

ModuleNotFoundError: No module named 'cv2'

## クロージャ？

In [93]:
def fibonacci_func():
    """フィボナッチ数列を返す関数を返す メモイズ機能つき"""
    table = {}  # 計算済みのフィボナッチ数列を格納するテーブル

    def fibonacci(n):
        # 計算したことのある数値についてはテーブルを参照して返す
        if n in table:
            return table[n]

        # 計算したことのない数値についてはフィボナッチ数列の定義どおり計算
        if n < 2:
            return 1
        table[n] = fibonacci(n - 1) + fibonacci(n - 2)
        return table[n]

    return fibonacci

for i in range(100):
    fib = fibonacci_func()
    print(fib(i))

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
39088169
63245986
102334155
165580141
267914296
433494437
701408733
1134903170
1836311903
2971215073
4807526976
7778742049
12586269025
20365011074
32951280099
53316291173
86267571272
139583862445
225851433717
365435296162
591286729879
956722026041
1548008755920
2504730781961
4052739537881
6557470319842
10610209857723
17167680177565
27777890035288
44945570212853
72723460248141
117669030460994
190392490709135
308061521170129
498454011879264
806515533049393
1304969544928657
2111485077978050
3416454622906707
5527939700884757
8944394323791464
14472334024676221
23416728348467685
37889062373143906
61305790721611591
99194853094755497
160500643816367088
259695496911122585
420196140727489673
679891637638612258
1100087778366101931
1779979416004714189
2880067194370816120
4660046610375530309
7540113804746346429
